In [1]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta
from hashlib import sha256
import codecs

from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer, auto_fit_columns
from bulletin.commom import static
from bulletin.commom.normalize import *

pd.set_option('display.max_columns', None)

timer = Timer()

In [2]:
cc = CasosConfirmados()
casos = cc.get_casos()

In [3]:
obitos_curitiba = pd.read_excel(join('input','obitos_curitiba.xlsx'),
                converters = {
                    'paciente': normalize_text,
                    'idade': lambda x: normalize_number(x,fill=0),
                    'mun_resid': normalize_text,
                    'rs': lambda x: str(normalize_number(x,fill=99)).zfill(2) if x != 99 else None
                },
                parse_dates=['data_cura_obito'])

obitos_curitiba['hash'] = obitos_curitiba.apply(lambda row: normalize_hash(row['paciente'])+str(row['idade'])+normalize_hash(row['mun_resid']), axis=1)
obitos_curitiba['hash_less'] = obitos_curitiba.apply(lambda row: normalize_hash(row['paciente'])+str(row['idade']-1)+normalize_hash(row['mun_resid']), axis=1)
obitos_curitiba['hash_more'] = obitos_curitiba.apply(lambda row: normalize_hash(row['paciente'])+str(row['idade']+1)+normalize_hash(row['mun_resid']), axis=1)

In [4]:
obitos_curitiba = obitos_curitiba.loc[ ~(obitos_curitiba['hash'].isin(casos['hash'])) ]
obitos_curitiba = obitos_curitiba.loc[ ~(obitos_curitiba['hash'].isin(casos['hash_less'])) ]
obitos_curitiba = obitos_curitiba.loc[ ~(obitos_curitiba['hash'].isin(casos['hash_more'])) ]

In [7]:
writer = pd.ExcelWriter(join('input','new_obitos_curitiba.xlsx'),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

obitos_curitiba.drop(columns=['hash','hash_less','hash_more']).to_excel(writer,sheet_name='obitos',index=None)
worksheet = writer.sheets['obitos']
auto_fit_columns(worksheet,obitos_curitiba)

writer.save()
writer.close()